Gets self efficacy and response efficacy score

Preprocessing
==============

Efficacy scores were collected in:
- every 3 days evening questionnaires (!! bug in the plateform, question was presented every day for some participants -> need to work out which and take every 3 day measures)

- baseline and followup efficacy - for EEG participants

- screening and followup questionnaire - get scores for online participants

- Participants who did the baseline and followup Efficacy score on the same survey as the every 3 days one: 
562, 477, 703, 786

In [1]:
library(tidyverse)
library(here)
library(lubridate)

#get filepaths

#eff_evening_filepath = here("Every3days")

cond_file = "~/Documents/scripts-local/skyline/BEHAVIOUR/Analysis/expe_condition_final.csv"


save_figure = here::here("Figures")

#read files

files_eff_eve <-dir(path=here::here("data/Every3days") , pattern = "*.csv") 
files_eff_bf <-dir(path=here::here("data/BaselineFollowup") , pattern = "*.csv") 
files_eff_online<-dir(path=here::here("data/OnlineFollowup") , pattern = "*.csv") 
# use here the data from the pp recruitment files
files_eff_base_online<-dir(path=here::here("data/OnlineBaseline") , pattern = "baseline_eff_score.csv") # sheet with screening ID
files_study_id<-dir(path=here::here("data/OnlineBaseline") , pattern = "study_ids_online.csv") # sheet with screening_id/study_ids corresponsdence
files_qualtrics<-dir(path=here::here("data/OnlineBaseline") , pattern = "qualtrics.csv") # sheet with screening_id/study_ids corresponsdence


every3days_scores <-read.csv(here::here("data/Every3days", files_eff_eve), header = TRUE, sep = ",", dec = ".", stringsAsFactors = FALSE)
baseline_followup_eeg <-read.csv(here::here("data/BaselineFollowup", files_eff_bf), header = TRUE, sep = ",", dec = ".", stringsAsFactors = FALSE)
followup_online <-read.csv(here::here("data/OnlineFollowup", files_eff_online), header = TRUE, sep = ",", dec = ".", stringsAsFactors = FALSE)
baseline_online <-read.csv(here::here("data/OnlineBaseline", files_eff_base_online), header = TRUE, sep = ",", dec = ".", stringsAsFactors = FALSE)
study_id <-read.csv(here::here("data/OnlineBaseline",files_study_id), header = TRUE, sep = ",", dec = ".", stringsAsFactors = FALSE)
qualtrics <-read.csv(here::here("data/OnlineBaseline",files_qualtrics), header = TRUE, sep = ",", dec = ".", stringsAsFactors = FALSE)

# condition file
cond <- read_csv(cond_file)


#Associate online pp baseline efficacy score with their study id:


#select only screening ID and qualtrics ID on recruitment file
pp_ids<-baseline_online %>% select("Screening.ID", "Response.ID") 

qualtrics<-rename(qualtrics, Response.ID = ResponseId)


remove(baseline_online)

baseline_online<-left_join(pp_ids, qualtrics, by= 'Response.ID')

# join with study id
baseline_online <- left_join(study_id, baseline_online)

# remove screening id
baseline_online<-select(baseline_online, -Screening.ID)



── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.4
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

here() starts at /home/claire/Documents/scripts-local/skyline/BEHAVIOUR/Analysis/Efficacy_scores


Attaching package: ‘lubridate’


The following object is masked from ‘package:here’:

    here


The following object is masked from ‘package:base’:

    date


Parsed with column specification:
cols(
  `Subject ID` = col_double(),
  day_msg = col_double(),
  `Starting Date` = col_character(),
  Good = col_double(),
  `Condition = Intervention` = col_double(),
  EEG = col_double(),
  Finished = col_double(),
  REMOVED = col_double(),
  Interview = col_double(),
  Notes = col_character(),
  `No Compliance` = co

In [2]:
#rename columns 
#- questions scoring: 0 = not at all; 100 = very much

every3days_scores<-rename(every3days_scores , date=EndDate, completed=Finished,
                  ID = Q1, 
                  r_eff1 = Q3_1, r_eff2 = Q3_2, 
                  s_eff1= Q4_1, s_eff2 = Q4_2, s_eff3= Q4_3, 
                  hope = Q7_1, 
                  quit_int = Q5
                  )

every3days_scores$ID<-as.factor(every3days_scores$ID)


baseline_followup_eeg <-rename(baseline_followup_eeg, date=EndDate, completed=Finished,
                  ID = Q1, 
                  r_eff1 = Q3_1, r_eff2 = Q3_2, 
                  s_eff1= Q4_1, s_eff2 = Q4_2, s_eff3= Q4_3, 
                  hope = Q7_1
                 # quit_int = Q5
                  )

baseline_followup_eeg$ID<-as.factor(baseline_followup_eeg$ID)


followup_online<-rename(followup_online, date=EndDate, completed=Finished, 
                      ID= Q1, 
                       r_eff1 = Q88_1, r_eff2= Q88_2, 
                       s_eff1 = Q90_1, s_eff2 = Q90_2, s_eff3= Q90_3, 
                       hope=Q92_1
                       )
followup_online$ID<-as.factor(followup_online$ID)

baseline_online<-rename(baseline_online, date=EndDate, completed=Finished, 
                       ID= Study.ID, 
                       r_eff1 = Q108_1, r_eff2= Q108_2, 
                       s_eff1 = Q110_1.1, s_eff2 = Q110_2.1, s_eff3= Q110_3.1, 
                       hope=Q112_1.1
                           )

            
baseline_online$ID<-as.factor(baseline_online$ID)

#Create one long dataset collating all data

# add a "session" column

every3days_scores <- add_column(every3days_scores, "type" = "every3days")
followup_online <- add_column(followup_online, "type" = "followup")
baseline_online<- add_column(baseline_online, "type" = "baseline")

#get sessions for the baseline-followup dataset
  

# remove extra header
#baseline_followup_eeg<-tail(baseline_followup_eeg, -2)

In [3]:
# create session based on occurrence of ID 
baseline_followup_eeg<-baseline_followup_eeg %>% 
        #mutate(date=ymd_hms(baseline_followup_eeg$date))  %>% 
        group_by(ID)  %>%   
        mutate('type'= row_number()) %>% 
        ungroup()
        
# assign factor and recode        
baseline_followup_eeg$type<-as.factor(baseline_followup_eeg$type)        
baseline_followup_eeg$type<-recode(baseline_followup_eeg$type, '1'="baseline", '2'="followup")                            


# select only columns of interest

baseline_followup_eeg<-select(baseline_followup_eeg, ID, date, r_eff1:hope, type, completed)
every3days_scores <-select(every3days_scores, ID, date, r_eff1:hope, type, completed)
followup_online<-select(followup_online, ID, date, r_eff1:hope, type, completed)
baseline_online<-select(baseline_online, ID, date, r_eff1:hope, type, completed)

# get only completed surveys
#every3days_scores<-filter(every3days_scores, completed=='True')

#baseline_followup_eeg<-filter(baseline_followup_eeg, completed=='True')
#
#followup_online<-filter(followup_online, completed=='True')
#baseline_online<-filter(baseline_online, completed=='True')

In [4]:
#Do some custom change to fit special cases


#---------------------------------#
# get data for the 4 pps who did the baseline and followup efficacy measure in the every3days

ID_add_to_bf <-c(562, 477, 703, 786)

tmp<- every3days_scores %>% 
  filter(ID %in% ID_add_to_bf)

tmp_bf_score <- tmp %>%  # get baseline and follow up scores - 1st and last 
  group_by(ID) %>% 
  slice(c(1, n())) %>% 
  ungroup()  

baseline_followup_eeg<-full_join(baseline_followup_eeg, tmp_bf_score)  # add to df_bf

# remove first and last measures then put back into main df dataframe, remove these ID from df before !

Joining, by = c("ID", "date", "r_eff1", "r_eff2", "s_eff1", "s_eff2", "s_eff3", "hope", "type", "completed")

Warning message:
“Column `ID` joining factors with different levels, coercing to character vector”
Warning message:
“Column `type` joining factor and character vector, coercing into character vector”


In [5]:
tmp_df <- tmp %>% 
  group_by(ID) %>% 
  slice(2:(n()-1)) %>% 
  ungroup()  


# remove those ID from df then put them back

every3days_scores<- every3days_scores %>% 
  filter(!ID %in% ID_add_to_bf )

every3days_scores<-full_join(every3days_scores, tmp_df)

rm(tmp, tmp_bf_score, tmp_df)

Joining, by = c("ID", "date", "r_eff1", "r_eff2", "s_eff1", "s_eff2", "s_eff3", "hope", "type", "completed")



In [27]:
# get IDs of pp who did the survey every day instead of every 3 days

# get date format
every3days_scores$date<-ymd_hms(every3days_scores$date)

# first remove  false alarms (non completed surveys)

every3days_scores<- every3days_scores %>% filter(every3days_scores$completed=='True')

# add a counter for each row per ID (see 1.)
every3days_scores <- every3days_scores %>% 
  group_by(ID) %>% 
  mutate(n_day = row_number()) %>%
  select(ID, n_day, everything()) %>% 
  ungroup()  
every3days_scores$n_day<-as.factor(every3days_scores$n_day)

#get_ID_bug <- every3days_scores %>% 
#  group_by(ID) %>% 
#  mutate(count = row_number()) %>% 
#  filter(count>9) %>% 
#  distinct(ID) %>% 
#  ungroup()  

In [65]:
# get pp who were in bug window
# get survey completed at start date + 3 +3 etc...

In [68]:
# get the experiment condition file with all pp IDs
cond <-rename(cond, ID = "Subject ID", condition = "Condition = Intervention", removed= "REMOVED")
cond$ID<-as.factor(cond$ID)


In [74]:
every3days_scores_tmp<-full_join(cond, every3days_scores)

Joining, by = "ID"

Warning message:
“Column `ID` joining factors with different levels, coercing to character vector”


In [146]:
# get list of pp who started during bug time window (July/August) 
#ID_bug<-every3days_scores_tmp %>% 
#            filter(n_day==1) %>% 
#            filter(date>=as.Date("2019-07-23") & date<=as.Date("2019-08-30") ) 



In [151]:
every3days_scores_tmp <-rename(every3days_scores_tmp , start=`Starting Date`)
every3days_scores_tmp$start<-dmy(every3days_scores_tmp$start)


In [157]:
df_bug<- every3days_scores_tmp %>% 
  filter(start>=as.Date("2019-07-20") & date<=as.Date("2019-08-30")) 


In [182]:
# get unique start dates
all_starts<-unique(df_bug$start)
all_starts

[1] "2019-07-20" "2019-07-23" "2019-07-25" "2019-07-27" "2019-08-10"
[6] "2019-08-09"

In [181]:
# get data for 1st date
df1<-df_bug %>% filter(start==all_starts[1])   
# get vector with dates corresponding to start date + every 3 days
n_dates<-vector()
for(i in seq(from=3, to=28, by=3)){
    date<-all_starts[1]+days(i)
    n_dates<-append(dates, date)
    }

# get corresponding data
df1 %>% filter(as.Date(date) %in% n_dates)

ID,day_msg,start,Good,condition,EEG,Finished,removed,Interview,Notes,⋯,n_day,date,r_eff1,r_eff2,s_eff1,s_eff2,s_eff3,hope,type,completed
<chr>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,⋯,<fct>,<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
270,89,2019-07-20,1,0,0,1,0,NA,took follow up questionnaire 1 month after end of its experiment,⋯,1,2019-07-23 10:12:08,100,100,34,0,21,31,every3days,True
270,89,2019-07-20,1,0,0,1,0,NA,took follow up questionnaire 1 month after end of its experiment,⋯,3,2019-07-26 21:08:28,100,55,0,0,20,24,every3days,True
270,89,2019-07-20,1,0,0,1,0,NA,took follow up questionnaire 1 month after end of its experiment,⋯,7,2019-07-29 21:23:51,100,100,0,0,24,1,every3days,True
270,89,2019-07-20,1,0,0,1,0,NA,took follow up questionnaire 1 month after end of its experiment,⋯,16,2019-08-07 22:43:46,100,100,0,0,0,0,every3days,True
270,89,2019-07-20,1,0,0,1,0,NA,took follow up questionnaire 1 month after end of its experiment,⋯,20,2019-08-10 21:27:21,100,100,0,0,8,0,every3days,True
270,89,2019-07-20,1,0,0,1,0,NA,took follow up questionnaire 1 month after end of its experiment,⋯,22,2019-08-13 11:40:32,100,100,0,0,0,0,every3days,True
270,89,2019-07-20,1,0,0,1,0,NA,took follow up questionnaire 1 month after end of its experiment,⋯,23,2019-08-13 21:05:33,100,100,1,10,19,0,every3days,True
270,89,2019-07-20,1,0,0,1,0,NA,took follow up questionnaire 1 month after end of its experiment,⋯,24,2019-08-16 12:42:53,100,100,0,9,9,0,every3days,True
209,86,2019-07-20,1,1,0,1,0,NA,NA,⋯,3,2019-07-26 21:23:19,60,70,60,54,78,57,every3days,True


In [ ]:
# make a loop from n=1 to len(all_starts)
for n in 1:length(all_starts){
    # get data for 1st date
    df2<-df_bug %>% filter(start==all_starts[n])   
    # get vector with dates corresponding to start date + every 3 days
    n_dates<-vector()
    for(i in seq(from=3, to=28, by=3)){
        date<-all_starts[n]+days(i)
        n_dates<-append(dates, date)
        }

    # get corresponding data
    df2 %>% filter(as.Date(date) %in% n_dates)
    }

In [ ]:
  
# for those IDs, only select data from the 1, 6, 9, 12, 15, 18, 21, 24, 27 days, and rename 1:9   
days<-c(1, 6, 9, 12, 15, 18, 21, 24, 27)

# create a new dataframe combining all pps 
df9<- every3days_scores %>% 
  filter(ID %in%get_ID_bug$ID & n_day %in% days) 
  

In [ ]:
df9$n_day<-recode_factor(df9$n_day, '1' = "1", '6'="2", '9'="3", '12' ="4", '15'="5", '18'="6", '21'="7", '24'="8", '27'="9")

tmp <- every3days_scores %>% 
filter(!ID %in% (get_ID_bug$ID) )

df9<- rbind(df9, tmp)

# create n_day column for baseline and followup
followup_online <- followup_online %>% mutate(n_day = '10')

baseline_online <- baseline_online %>% mutate(n_day = '0')

baseline_followup_eeg<-baseline_followup_eeg %>% 
    mutate(n_day=ifelse(type=='baseline', '0', '10'))
    

#baseline_online<-filter(baseline_online, completed=='True')



# remove extra every 3 days rows
baseline_followup_eeg<-baseline_followup_eeg %>% filter(type=='baseline'|type=='followup')


eff_scores<-bind_rows(baseline_followup_eeg, df9)


tmp<-bind_rows(eff_scores, followup_online) 

tmp2<- bind_rows(tmp, baseline_online)


remove(eff_scores)
eff_scores<-tmp2


eff_scores<-tail(eff_scores, -2) # remove extra header
eff_scores$ID<-as.factor(eff_scores$ID)
eff_scores$type<-as.factor(eff_scores$type)

eff_scores %>% 
        group_by(ID)  %>%   
        mutate(check= row_number()) %>% 
        ungroup()
        

Create one big dataframe assembling all the others

In [ ]:
# associate with cond file 
eff_scores$ID<-as.factor(eff_scores$ID)


tmp<-full_join(cond, eff_scores)


remove(eff_scores)
eff_scores<-tmp



#recode condition to human names

eff_scores$condition<-recode(eff_scores$condition, '0' = "control", '1' = "intervention")

Filter participants based on survey completion criteria

In [ ]:
#Replace missing values and keep only participants who were not removed
eff_scores$removed<-eff_scores$removed %>% replace(is.na(.), 0)

In [ ]:
#filter to keep participants not removed
eff_scores<-eff_scores%>%filter(removed == 0)

In [ ]:
# filter to keep participants who did at least 80% of surveys
#eff_scores<-eff_scores%>%filter(Good == 1)

Do a check of for missing values in the data


In [ ]:
missing.values <- eff_scores %>%
    gather(key = "key", value = "val") %>%
    mutate(is.missing = is.na(val)) %>%
    group_by(key, is.missing) %>%
    summarise(num.missing = n()) %>%
    filter(is.missing==T) %>%
    select(-is.missing) %>%
    arrange(desc(num.missing)) 

missing.values

Look at why we have condition with missing values


In [ ]:
tmp<-which(is.na(eff_scores$condition))

In [ ]:
eff_scores[tmp, ]

In [ ]:
# remove these lines 
eff_scores<-drop_na(eff_scores, condition)


In [ ]:
which(is.na(eff_scores$s_eff1))

In [ ]:
eff_scores<-drop_na(eff_scores, r_eff1)


In [ ]:
df<-eff_scores


 Compute response and self efficacy scores:
--------------------------------------------

get the mean of all individual scores in each condition for each day:

- define days 1-2-3 etc.
- check for those pp who did baseline and followup (IDs 562, 477, 703, 786)
- find participants with a maximum of 9 n_day
- find participants with more than 9 n_day entries
- take only only every 3 days: 1, 6, 9, 12, 15, 18, 21

create new dataset:  
- count how many of each day per condition


In [ ]:
# transform the columns for response and self efficacy that were charactuers in numeric tyoe           
df <-df %>% mutate_each(funs(as.numeric), -ID,  -date, -condition, -type) 




In [ ]:
df<-select(df, -'removed', -'Finished', -'Interview', -'Notes', -"completed" )

In [ ]:
df<-select(df, -'Starting Date', -'day_msg',  -'No Compliance', -'Email reminder poor perf', -'eve_msg', -'ev3days',  -'most_recent', -'NAME', -'EMAIL', -'PHONE')

In [ ]:
colnames(df)

In [ ]:
# pass n_days as factor and recode day 0 and 10
df$n_day<-as.factor(df$n_day)

df$n_day<-recode(df$n_day, '0'= 'baseline', '10'='follow-up' )

count how many of each day per condition

In [ ]:
n_rep <- df %>% group_by(condition, n_day, EEG) %>%
 summarize(count=n()) %>% 
    ungroup()
n_rep <-n_rep %>% drop_na()

n_rep$EEG<-as.factor(n_rep$EEG)


# plot
ggplot(data= n_rep, aes(x = n_day, y=count)) +
  geom_point(mapping = aes(colour = condition, shape =EEG ) ) +
  ggtitle("Number of responses per measures  - efficacy questionnaire")+
  xlab('Measure')+ ylab('number of responses')+ 
 ggsave('nresp_per_day.jpg', plot =last_plot(), dpi =300, path= save_figure)
 

Work on scores for self, response and hope

In [ ]:
colnames(df)

In [ ]:
# take the score of each (resp and self) item for each day for each pp

df<- df %>% 
         mutate(score_re= rowMeans(select(., c('r_eff1','r_eff2')), na.rm=TRUE) , 
         score_se= rowMeans(select(., c('s_eff1', 's_eff2', 's_eff3')), na.rm=TRUE), 
         score_hope= rowMeans(select(., c('hope')), na.rm=TRUE)        
         ) 
    

In [ ]:
df

In [ ]:
df2<-select(df, -"r_eff1", -"r_eff2", -"s_eff1", -"s_eff2", -"s_eff3", -"hope")

In [ ]:
#df2<- rename(df2, total_hope=hope)

In [ ]:
#df2<-select(df2, -'Good', -'EEG', -'date')

In [ ]:
head(df2)

In [ ]:
# turn df into long format 
df2_long<-df2 %>% 
  pivot_longer(
    -c('ID', 'condition','type', 'n_day', 'Good', 'EEG', 'date'),  
    names_to = c(".value", "value"), 
    names_sep = "_", 
    values_drop_na = TRUE
  )

In [ ]:
df2_long$EEG<-as.factor(df2_long$EEG)

In [ ]:
# compute summary statistics per condition per day for each score
summary_df2 <- df2_long %>%
  group_by(condition, n_day, value) %>%
  summarise(mean.score = mean(score),
            sd.score= sd(score), count = n(),
            se.mean = sd.score/sqrt(count), 
            ci.mean= qt(1-(1-0.95/2),count-1)*se.mean)

In [ ]:
summary_df2  %>% filter(n_day == 'baseline')

In [ ]:
head(summary_df2)


- 3. Compute mean response and self efficacy score for each day for each pp

- 4. plot
!!! different number of data points per paricipant !!!

# test plot

https://stackoverflow.com/questions/55262229/how-to-use-geom-errorbar-with-facet-wrap-in-ggplot2

In [ ]:
#write_csv(m_scores, "summary_stats.csv")

Plot response and self efficacy scores

In [ ]:
test_eff<-summary_df2 %>% 
        filter(value=="re" | value=="se") 

In [ ]:
gs1 <- ggplot(test_eff, aes(x=n_day, y=mean.score)) + 
       geom_point(mapping = aes(colour=condition, size=count), alpha=1/3)+ 
       facet_wrap(~value)+
       geom_errorbar(aes(ymin = mean.score - ci.mean, ymax = mean.score + ci.mean), 
                width=0.2)+
    
        #geom_smooth(method=lm, se=FALSE, fullrange=TRUE)+
  xlab("Measures")+ylab("Mean score")+
    ggtitle("Mean scores for response and self efficacy per day per condition,\n reported every 3 days and at baseline and followup")
gs1 + theme(panel.spacing.x=unit(2, "cm"),panel.spacing.y=unit(1, "lines")) +
 theme(axis.text.x = element_text(angle = 90, hjust = 1))
ggsave('efficacy_scores.jpg', plot =last_plot(), dpi =300, path= save_figure)

Seems like scores were quite high to start with for response efficacy, maybe a plateau effect while self efficacy scores were lower at baseline

Plot hope score

In [ ]:
test_hope<-summary_df2 %>% 
        filter(value=="hope")  

In [ ]:
test_hope

In [ ]:

ggplot(data=test_hope, mapping=aes(x= n_day, y=mean.score)) +
      geom_point(mapping = aes(colour=condition, size=count), alpha=1/3)+ 
       geom_errorbar(aes(ymin = mean.score - ci.mean, ymax = mean.score + ci.mean), 
                width=0.2)+
    
        #geom_smooth(method=lm, se=FALSE, fullrange=TRUE)+
  xlab("Measures")+ylab("Mean score")+
   ggtitle("Mean scores for how hopeful about quitting smoking")

  ggsave('hope_scores_v2.jpg', plot =last_plot(), dpi =300, path= save_figure)


Make raincloud plots

In [ ]:
library(cowplot)
library(readr)

source("~/Documents/RainCloudPlots/tutorial_R/R_rainclouds.R")
source("~/Documents/RainCloudPlots/tutorial_R/summarySE.R")

theme_set(theme_cowplot())

In [ ]:
packages <- c("ggplot2", "dplyr", "lavaan", "plyr", "cowplot", "rmarkdown",
"readr", "caTools", "bitops")
if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
install.packages(setdiff(packages, rownames(installed.packages())))
}

In [ ]:
df3<-df2

In [ ]:
which(is.na(df3$n_day))

In [ ]:
colnames(df3)

In [ ]:
df3<-drop_na(df3, n_day)

In [ ]:
 ggplot(df3, aes(x = n_day, y = score_se, fill = condition)) +
  geom_flat_violin(aes(fill = condition),position = position_nudge(x = .1, y = 0), adjust = 1.5, trim = FALSE, alpha = .5, colour = NA)+
  geom_point(aes(x = as.numeric(n_day)-.15, y = score_se, colour = condition),position = position_jitter(width = .05), size = .25, shape = 20)+
  geom_boxplot(aes(x = n_day, y = score_se,  fill = condition),outlier.shape = NA, alpha = .5, width = .1, colour = "black")+
 # geom_line(data = sumrepdat, aes(x = as.numeric(condition)+.1, y = score_mean, group = time, colour = time), linetype = 3)+
 # geom_point(data = sumrepdat, aes(x = as.numeric(group)+.1, y = score_mean, group = time, colour = time), shape = 18) +
 # geom_errorbar(data = sumrepdat, aes(x = as.numeric(group)+.1, y = score_mean, group = time, colour = time, ymin = score_mean-se, ymax = score_mean+se), width = .05)+
  scale_colour_brewer(palette = "Dark2")+
  scale_fill_brewer(palette = "Dark2")+
  xlab('Measure') + ylab('Score')+
  ggtitle("Self-efficacy score") +
  coord_flip()

ggsave('raincloud_self_eff_scores.jpg', plot =last_plot(), dpi =300, path= save_figure)


In [ ]:
ggplot(df3, aes(x = n_day, y = score_re, fill = condition)) +
  geom_flat_violin(aes(fill = condition),position = position_nudge(x = .1, y = 0), adjust = 1.5, trim = FALSE, alpha = .5, colour = NA)+
  geom_point(aes(x = as.numeric(n_day)-.15, y = score_re, colour = condition),position = position_jitter(width = .05), size = .25, shape = 20)+
  geom_boxplot(aes(x = n_day, y = score_re,  fill = condition),outlier.shape = NA, alpha = .5, width = .1, colour = "black")+
 # geom_line(data = sumrepdat, aes(x = as.numeric(condition)+.1, y = score_mean, group = time, colour = time), linetype = 3)+
 # geom_point(data = sumrepdat, aes(x = as.numeric(group)+.1, y = score_mean, group = time, colour = time), shape = 18) +
 # geom_errorbar(data = sumrepdat, aes(x = as.numeric(group)+.1, y = score_mean, group = time, colour = time, ymin = score_mean-se, ymax = score_mean+se), width = .05)+
  scale_colour_brewer(palette = "Dark2")+
  #scale_colour_manual(values=cbPalette)+
  #scale_fill_manual(values=cbPalette)+
  scale_fill_brewer(palette = "Dark2")+
  xlab('Measure') + ylab('Score')+
  ggtitle("Response-efficacy score") +
  coord_flip()

ggsave('raincloud_resp_eff_scores.jpg', plot =last_plot(), dpi =300, path= save_figure)


In [ ]:
ggplot(df3, aes(x = n_day, y = score_hope, fill = condition)) +
  geom_flat_violin(aes(fill = condition),position = position_nudge(x = .1, y = 0), adjust = 1.5, trim = FALSE, alpha = .5, colour = NA)+
  geom_point(aes(x = as.numeric(n_day)-.15, y = score_hope, colour = condition),position = position_jitter(width = .05), size = .25, shape = 20)+
  geom_boxplot(aes(x = n_day, y = score_hope,  fill = condition),outlier.shape = NA, alpha = .5, width = .1, colour = "black")+
 # geom_line(data = sumrepdat, aes(x = as.numeric(condition)+.1, y = score_mean, group = time, colour = time), linetype = 3)+
 # geom_point(data = sumrepdat, aes(x = as.numeric(group)+.1, y = score_mean, group = time, colour = time), shape = 18) +
 # geom_errorbar(data = sumrepdat, aes(x = as.numeric(group)+.1, y = score_mean, group = time, colour = time, ymin = score_mean-se, ymax = score_mean+se), width = .05)+
  scale_colour_brewer(palette = "Dark2")+
  scale_fill_brewer(palette = "Dark2")+
  xlab('Measure') + ylab('Score')+
  ggtitle("Hope score") +
  coord_flip()

ggsave('raincloud_hope_scores.jpg', plot =last_plot(), dpi =300, path= save_figure)


In [ ]:
colnames(df3)